In [ ]:
####

In [ ]:
import gym 
import collections

In [ ]:
class Config:
    def __init__(self):
        self.GAMMA = 0.9
        self.ENV_NAME = 'FrozenLake-v0'
        self.TEST_EPISODES = 20

In [ ]:
class Agent:
    def __init__(self):
        self.config = Config()

        self.env = gym.make(self.config.ENV_NAME)
        self.state = self.env.reset()

        self.reward = collections.defaultdict(float)
        self.transists = collections.defaultdict(collections.Counter)
        self.values = collections.defaultdict(float)

    def play_n_random_steps(self , steps):
        for _ in range(steps):
            action = self.env.action_space.sample()
            new_state , reward , is_done , _ = self.env.step(action)

            self.reward[(self.state , action , new_state)] = reward
            self.transists[(self.state , action)][new_state] += 1
            
            self.state = self.env.reset() if is_done else new_state
    
    def calc_action_value(self , state , action):
        target_counts = self.transists[(state , action)]
        total = sum(target_counts.values())
        action_values = 0.0

        for tgt , count in target_counts.items():
            reward = self.reward[(state , action , tgt)]
            val = reward + self.config.GAMMA * self.values[tgt]
            action_values += (count / total) * val
        return action_values

    def select_action(self , state):
        best_action , best_value = None , None

        for action in range(self.env.action_space.n):
            action_value = self.calc_action_value(state , action)
            if best_action is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_action

    def play_episode(self , env):
        total_reward = 0.0
        state = env.reset()

        while True:
            action = self.select_action(state)
            new_state , reward , is_done , _ = env.step(action)
            
            self.reward[(state , action , new_state)] = reward
            self.transists[(state , action)][new_state] += 1
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward

    def value_iteration(self):
        for state in range(self.env.observation_space.n):
            state_values = [self.calc_action_value(state , action) for action in range(self.env.action_space.n)]
            self.values[state] = max(state_values)

In [ ]:
config = Config()
agent = Agent()
env = gym.make(config.ENV_NAME)

In [ ]:
iter_no = 0
best_reward = 0.0

while True:
    iter_no += 1

    agent.play_n_random_steps(100)
    agent.value_iteration()
    reward = 0.0

    for _ in range(config.TEST_EPISODES):
        reward += agent.play_episode(env)
    reward /= config.TEST_EPISODES

    if reward > best_reward:
        print(f'Best reward updated {best_reward} , {reward}')
        best_reward = reward
    if reward > 0.80:
        print(f'Solved in iterations {iter_no}')
        break

Best reward updated 0.0 , 0.6
Best reward updated 0.6 , 0.75
Best reward updated 0.75 , 0.9
Solved in iterations 32
